In [1]:
import numpy as np
from get_border_cases import get_border_cases
import matplotlib as mpl
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
n = 17
N = n - 1
inner_n = n - 2
border_cases = get_border_cases(N)
params = np.ones(inner_n ** 2)
try_num = 3
output_absorption_times = f"..\\output\\eps_quarter\\pvp\\try_{try_num}\\"
output_absorption_images1 = f"..\\output\\eps_quarter\\pvp\\try_{try_num}\\absorption_times\\"
output_absorption_images2 = f"..\\output\\eps_quarter\\pvp\\try_{try_num}\\center_strategies\\"
output_absorption_images3 = f"..\\output\\eps_quarter\\pvp\\try_{try_num}\\border_strategies\\"
output_duration = f"..\\output\\eps_quarter\\pvp\\try_{try_num}\\"

In [3]:
def find_max(func, game):
    if func(0, game) >= func(1, game):
        v = func(0, game)
        p = 0
    else:
        v = func(1, game)
        p = 1
    return v, p

def get_win(p, game):
    return (p / 2) * (game[0, 0] + game[0, 1]) + (1 - p) / 2 * (game[1, 0] + game[1, 1])

'''
    Computes the value of a game.
    game is 2x2 matrix.
'''
def get_value(game):
    return (game[0, 0] * game[1, 1] - game[0, 1] * game[1, 0]) / (game[0, 0] + game[1, 1] - game[1, 0] - game[0, 1])

'''
    Compute optimal strategy for player 1.
'''
def get_p1(game):
    return (game[1, 1] - game[1, 0]) / (game[0, 0] + game[1, 1] - game[1, 0] - game[0, 1])

'''
    Compute optimal strategy for player 2.
'''
def get_q1(game):
    return (game[1, 1] - game[0, 1]) / (game[0, 0] + game[1, 1] - game[1, 0] - game[0, 1])

'''
    Converts global index to inner index.
    index - index to convert.
    inner_n - inner dimension (without borders).
    global_n - outer dimension (with borders).
'''
def inner_n_to_global_N(index, inner_n, global_n):
    row = index // inner_n
    col = index % inner_n
    global_index = (row + 1) * global_n + (col + 1)
    return global_index

def global_N_to_inner_n(index, inner_n, global_n):
    row = index // global_n
    col = index % global_n

    if row < 1 or col < 1 or row > global_n - 2 or col > global_n - 2:
        raise ValueError("Index should match an inner node.")
    inner_index = (row - 1) * inner_n + (col - 1)
    return inner_index

'''
    Checks if a game matrix has saddles.
'''
def check_for_saddle(game):
    maximin = np.max(np.min(game, axis=1))
    minimax = np.min(np.max(game, axis=0))
    res = 'N'
    if np.isclose(maximin, minimax, rtol=1e-8, atol=1e-8):
        res = maximin
        index = next(i for i, _ in enumerate(game.flatten()) if np.isclose(_, res, rtol=1e-8, atol=1e-8)) 
        return res, (1 - index // 2, 1 - index % 2) # fixed
    return 'N'

def get_quarter(index):
    row = index // inner_n
    col = index % inner_n
    center_row = inner_n // 2
    center_col = inner_n // 2
    if row == center_row and col == center_col:
        return 0
    elif row == center_row:
        if col > center_col:
            return 8  # Правая часть горизонтальной оси
        else:
            return 4  # Левая часть горизонтальной оси
    elif col == center_col:
        if row < center_row:
            return 2  # Верхняя часть вертикальной оси
        else:
            return 6  # Нижняя часть вертикальной оси
    elif row < center_row:
        if col < center_col:
            return 3  # Вторая четверть
        else:
            return 1  # Первая четверть
    else:
        if col < center_col:
            return 5  # Третья четверть
        else:
            return 7  # Четвертая четверть

def compute_epsilon_playerBorder(index, epsilon, direction):
    quarter = get_quarter(index)
    if quarter == 1:
        if direction in ['left', 'down']:
            return -epsilon
        else:
            return epsilon
    elif quarter == 3:
        if direction in ['right', 'down']:
            return -epsilon
        else:
            return epsilon
    elif quarter == 5:
        if direction in ['right', 'up']:
            return -epsilon
        else:
            return epsilon
    elif quarter == 7:
        if direction in ['left', 'up']:
            return -epsilon
        else:
            return epsilon
    elif quarter == 2:
        if direction == 'down':
            return -epsilon
        else:
            return epsilon
    elif quarter == 4:
        if direction == 'right':
            return -epsilon
        else:
            return epsilon
    elif quarter == 6:
        if direction == 'up':
            return -epsilon
        else:
            return epsilon
    elif quarter == 8:
        if direction == 'right':
            return -epsilon
        else:
            return epsilon
    else:
        return 0

def compute_epsilon_playerCenter(index, epsilon, direction):
    quarter = get_quarter(index)
    if quarter == 1:
        if direction in ['left', 'down']:
            return epsilon
        else:
            return -epsilon
    elif quarter == 3:
        if direction in ['right', 'down']:
            return epsilon
        else:
            return -epsilon
    elif quarter == 5:
        if direction in ['right', 'up']:
            return epsilon
        else:
            return -epsilon
    elif quarter == 7:
        if direction in ['left', 'up']:
            return epsilon
        else:
            return -epsilon
    elif quarter == 2:
        if direction == 'down':
            return epsilon
        else:
            return -epsilon
    elif quarter == 4:
        if direction == 'right':
            return epsilon
        else:
            return -epsilon
    elif quarter == 6:
        if direction == 'up':
            return epsilon
        else:
            return -epsilon
    elif quarter == 8:
        if direction == 'right':
            return epsilon
        else:
            return -epsilon
    else:
        return 0
# q == 0.5
def get_game(index, w, epsilon):
    game = np.zeros((2, 2))
    game[0, 0] = compute_a11(index, w, epsilon)
    game[0, 1] = compute_a12(index, w, epsilon)
    game[1, 0] = compute_a21(index, w, epsilon)
    game[1, 1] = compute_a22(index, w, epsilon)
    return game
# Up
def compute_a11(index, w, epsilon):
    if (index - n) in border_cases:
        a11 = 1
    else:
        adjusted_epsilon = compute_epsilon_playerCenter(index, epsilon, 'up')
        adjusted_epsilon += compute_epsilon_playerBorder(index, epsilon, 'up')
        a11 = w[global_N_to_inner_n(index - n, inner_n, n)] + 1 + adjusted_epsilon
    return a11
    
# Down
def compute_a21(index, w, epsilon):
    if (index + n) in border_cases:
        a12 = 1
    else:
        adjusted_epsilon = compute_epsilon_playerCenter(index, epsilon, 'down')
        adjusted_epsilon += compute_epsilon_playerBorder(index, epsilon, 'down')
        a12 = w[global_N_to_inner_n(index + n, inner_n, n)] + 1 + adjusted_epsilon
    return a12

# Right
def compute_a12(index, w, epsilon):
    if (index + 1) in border_cases:
        a21 = 1
    else:
        adjusted_epsilon = compute_epsilon_playerCenter(index, epsilon, 'right')
        adjusted_epsilon += compute_epsilon_playerBorder(index, epsilon, 'right')
        a21 = w[global_N_to_inner_n(index + 1, inner_n, n)] + 1 + adjusted_epsilon
    return a21

# Left
def compute_a22(index, w, epsilon):
    if (index - 1) in border_cases:
        a22 = 1
    else:
        adjusted_epsilon = compute_epsilon_playerCenter(index, epsilon, 'left')
        adjusted_epsilon += compute_epsilon_playerBorder(index, epsilon, 'left')
        a22 = w[global_N_to_inner_n(index - 1, inner_n, n)] + 1 + adjusted_epsilon
    return a22
    

'''
    Number of values in w should match number of equations.
'''
def prepare_equations(w, epsilon):
    eqs = np.zeros(len(w))
    p1s = []
    q1s = []
    #prev_distances = [compute_distance_to_center(inner_n_to_global_N(i, inner_n, n)) for i in range(len(w))]
    for i in range(len(w)):
        index = inner_n_to_global_N(i, inner_n, n)
        game_mx = get_game(index, w, epsilon)
        saddle = check_for_saddle(game_mx)
        
        if saddle == "N":
            v = get_value(game_mx)
            p1 = get_p1(game_mx)
            q1 = get_q1(game_mx)
        else:
            v, (p1, q1) = saddle

        p1s.append(p1)
        q1s.append(q1)
        eqs[i] = w[i] - v

    return tuple(eqs)

In [4]:
from scipy.optimize import fsolve
from tqdm import tqdm

epsilon_values = np.arange(0.0, 11.0, 1.0)
file = open("D:\RW\\RWAnalyzer\\output\\eps_quarter\\pvp\\" + "epsilon_val_try_3.txt", "w")
w_new_list = []
for epsilon in tqdm(epsilon_values, desc='Solving equations'):
    file.write(f"{epsilon} ")
    message = ""
    while (message != "The solution converged."):
        starting_params = np.random.random(inner_n ** 2) * (inner_n - 1) ** 2 * 0.68
        w_new, _, _, message  = fsolve(lambda w: prepare_equations(w, epsilon), tuple(starting_params), full_output=True)
        #previous_distances.clear()
    w_new_list.append(w_new)
    starting_params, message
file.close()

<>:5: SyntaxWarning: invalid escape sequence '\R'
<>:5: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Илья\AppData\Local\Temp\ipykernel_860\794433876.py:5: SyntaxWarning: invalid escape sequence '\R'
  file = open("D:\RW\\RWAnalyzer\\output\\eps_quarter\\pvp\\" + "epsilon_val_try_3.txt", "w")
Solving equations: 100%|██████████| 11/11 [39:26<00:00, 215.13s/it] 


In [5]:
step = 0
p1s = [[] for _ in range(len(epsilon_values))]
q1s = [[] for _ in range(len(epsilon_values))]
vs = [[] for _ in range(len(epsilon_values))]
for epsilon in epsilon_values:
    for i in range(len(w_new)):
        index = inner_n_to_global_N(i, inner_n, n)
        game_mx = get_game(index, w_new_list[step], epsilon)
        
        saddle = check_for_saddle(game_mx)
        
        if saddle == "N":
            v = get_value(game_mx)
            p1 = get_p1(game_mx)
            q1 = get_q1(game_mx)
        else:
            v, (p1, q1) = saddle

        p1s[step].append(p1)
        q1s[step].append(q1)
        vs[step].append(v)
    step += 1

In [6]:
from make_prob_matrix import make_prob_matrix
def make_banded_matrix(A, N):
    banded_matrix = np.zeros((2 * N + 1, N ** 2), dtype=np.float32)
    for i in [-N, -1, 0, 1, N]:
        d = np.diagonal(A, -i)
        if i < 0:
            banded_matrix[i + N, -i:] = d
        if i > 0:
            banded_matrix[i + N, :-i] = d
        if i == 0:
            banded_matrix[i + N, :] = d
    return banded_matrix

def find_mean_time_banded(A, N):
    banded_matrix = make_banded_matrix(np.eye(A.shape[0]) - A, N)
    import scipy
    mean_times = scipy.linalg.solve_banded((N, N), banded_matrix, np.ones(banded_matrix.shape[1]))
    return mean_times[N * (N // 2) + N // 2], mean_times

def find_mean_time(Q, N): # Sparse is slow
    mean_times = np.linalg.solve(np.eye(Q.shape[0]) - Q, np.ones(Q.shape[0]))
    return mean_times[N * (N // 2) + N // 2]

if not os.path.isdir(f"..\\output\\eps_quarter\\pvp\\try_{try_num}"):
    os.makedirs(output_absorption_images1, exist_ok=True)
    os.makedirs(output_absorption_images2, exist_ok=True)
    os.makedirs(output_absorption_images3, exist_ok=True)

#print(len(p1s))
#print(len(q1s))
#print(len(p1s[0]))
#print(len(q1s[0]))
step = 0
file = open(output_duration + "duration.txt", "w")
p1_matrices = []
q1_matrices = []
for epsilon in epsilon_values:
    p1_matrix = np.reshape(p1s[step], (15, 15))
    q1_matrix = np.reshape(q1s[step], (15, 15))
    p1_matrix_padded = np.pad(p1_matrix, pad_width=1, mode='constant', constant_values=0)
    #print("-------------------------")
    #print(p1_matrix_padded)
    #print("-------------------------")
    q1_matrix_padded = np.pad(q1_matrix, pad_width=1, mode='constant', constant_values=0)
    qr_optimal, probability_optimal = make_prob_matrix(N, p1_matrix_padded.T, q1_matrix_padded.T)
    mean_time, mean_times = find_mean_time_banded(probability_optimal, N - 1)
    file.write(f"{mean_time} ")
    #print(mean_times)
    #p1_matrices.append(p1_matrix)
    #q1_matrices.append(q1_matrix)
    step += 1
#qr_optimal, probability_optimal = make_prob_matrix(N, p1s, q1s)
#mean_time, mean_times = find_mean_time_banded(probability_optimal, N - 1)
file.close()
#print(mean_time)
#print('\n')
#print(mean_times)

In [7]:
step = 0
if not os.path.isdir(f"..\\output\\eps_quarter\\pvp\\try_{try_num}"):
    os.makedirs(output_absorption_images1, exist_ok=True)
    os.makedirs(output_absorption_images2, exist_ok=True)
    os.makedirs(output_absorption_images3, exist_ok=True)
#file = open("D:\RW\\pvb_2\\try_5\\absorption_time.txt", "w")

file = open(output_absorption_times + "absorption_time.txt", "w")
for epsilon in epsilon_values:
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    a = ax.imshow(np.reshape(vs[step], newshape=(inner_n, inner_n)))
    fig.colorbar(a)
    for (j, i), label in np.ndenumerate(np.reshape(vs[step], newshape=(inner_n, inner_n))):
        #print_str_1 = "↑→" if np.isclose(label, 1) else ""
        #print_str_2 = "↓←" if np.isclose(label, 0) else ""
        if i == 7 and j == 7:
            file.write(f"{label:.2f} ")
        ax.text(i, j, f"{label:.2f} ", ha='center', va='center')
        ax.text(i, j, f"{label:.2f} ", ha='center', va='center')
        #print(label)
    ax.set_title(f"Mean absorption time {epsilon:.4f}")
    str_ = f"pvp_strategy_{epsilon:.4f}.png"
    #fig.savefig("D:\RW\\pvb_2\\try_5\\1\\" + str_)
    fig.savefig(output_absorption_images1 + str_)
    plt.close()
    step += 1
file.close()

In [8]:
border_color_3 = np.array([204, 153, 255]) / 255.0
border_color_2 = np.array([178, 102, 255]) / 255.0
border_color_1 = np.array([141, 29, 255]) / 255.0
white_color = np.array([255, 255, 255]) / 255.0
center_color_1 = np.array([255, 202, 26]) / 255.0
center_color_2 = np.array([255, 219, 102]) / 255.0
center_color_3 = np.array([255, 231, 153]) / 255.0
rwgame_cmap = mcolors.LinearSegmentedColormap.from_list(
        'rwgame', [border_color_1, border_color_2, border_color_3, white_color, center_color_3, center_color_2, center_color_1])

In [9]:
step = 0
for epsilon in epsilon_values:
        font = {'family' : 'sans-serif',
                'size'   : 5}

        mpl.rc('font', **font)

        fig, ax = plt.subplots(1, 1, dpi=300)
        ax.invert_yaxis()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        a = ax.imshow(np.reshape(p1s[step], newshape=(inner_n, inner_n)), cmap=rwgame_cmap)
        ax.plot()

        for (j, i), label in np.ndenumerate(np.reshape(p1s[step], newshape=(inner_n, inner_n))):
                print_str_1 = "\n↑→" if np.isclose(label, 1) else ""
                print_str_2 = "\n↓←" if np.isclose(label, 0) else ""
                plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
                plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')

        font = {'family' : 'sans-serif',
                'size'   : 8}
        mpl.rc('font', **font)

        cbar = fig.colorbar(a)
        cbar.ax.text(4, 1, '↑→', ha='center', va='center')
        cbar.ax.text(4, 0, '↓←', ha='center', va='center')

        ax.set_title(f"Вероятность игрока \"за центр\" выбрать 1 стратегию, eps = {epsilon:.4f}.")
        str_ = f"pvp_2_strategy_{epsilon:.4f}.png"
        #fig.savefig("D:\RW\\pvb_2\\try_5\\3\\" + str_)
        fig.savefig(output_absorption_images2 + str_)
        plt.close()
        #fig.savefig("G:\Studies\Masters\img\\pvb_strategy.png")
        step += 1

In [10]:
step = 0
for epsilon in epsilon_values:
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    a = ax.imshow(np.reshape(q1s[step], newshape=(inner_n, inner_n)))
    fig.colorbar(a)

    for (j, i), label in np.ndenumerate(np.reshape(q1s[step], newshape=(inner_n, inner_n))):
        print_str_1 = "↑→" if np.isclose(label, 1) else ""
        print_str_2 = "↓←" if np.isclose(label, 0) else ""
        plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
        plt.text(i, j, f"{label:.2f} " + print_str_1 + print_str_2, ha='center', va='center')
    ax.set_title(f"Вероятность игрока \"за границу\" выбрать 1 стратегию, eps = {epsilon:.4f}.")
    str_ = f"pvp_strategy_{epsilon:.4f}.png"
    #fig.savefig("D:\RW\\pvb_2\\try_5\\2\\" + str_)
    fig.savefig(output_absorption_images3 + str_)
    plt.close()
    #fig.savefig("G:\Studies\Masters\img\\pvb_2_strategy.png")
    step += 1
